# KR260 DMA Capture Buffer

## Setup

### Mounting Development Machine
This notebook utilizes `sshfs` to mount the development directory on the KR260 board to avoid transferring files bewteen machines.  

The development machine that was used to build the KR260 FPGA image is named `dev-wks`.  To mount the development directory on `dev-wks`, first will need to create a folder to mount the directory, then execute the following from the Jupyter Terminal:

```bash
cd /home/root/jupyter_notebooks
mkdir puch
sshfs sdr@dev-wks:/home/sdr/workspace/puch-workspace/HLS-QPSK-Demod-Baseband_002 /home/root/jupyter_notebooks/puch
```

If `sshfs` is not installed, then execute `sudo apt install sshfs` on the Jupyter Terminal.

### Unmounting Development Machine
To unmount the directory
` fusermount -u /home/root/jupyter_notebooks/dev-wks/`


### Plot function for use in this notebook
The first code block below defines a function that we will use for plotting data throughout this notebook. Note that the function has a `n_samples` argument so that we can limit the number of samples to plot. Plotting more than a few thousand samples can be very slow and consume a lot of RAM.

### Requirements
Install the following:

```bash
pip install numpy==1.26.4
pip install ipympl
pip install ipywidgets
pip install jupyter_bokeh
```

# Initilize

#### Bokeh Plot Function

In [1]:
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook, show
import numpy as np
import math


def plot_time(in_signal,out_signal=None):
    t = np.linspace(0,len(in_signal),len(in_signal))
    output_notebook()
    p = figure(title = "Input & Output Signal")
    
    if out_signal is not None:
        p.line(t,out_signal,legend_label="Output Signal",line_color="red",line_width=1)
    p.line(t,in_signal,legend_label="Input Signal",line_color="blue",line_width=3)
    show(p)

#### Load Overlay

In [2]:
from pynq import Overlay
from pynq import allocate
from pynq import MMIO
import pynq.lib.dma
import time

#import Python library from repo:
import sys
sys.path.append('/root/jupyter_notebooks/puch/')
import fpga.py.puch as puch
import fpga.lib.timestamp.sw.timestamp_regmap as timestamp_regmap
import fpga.lib.led_reg.sw.led_regmap as led_regmap
import fpga.lib.DMA_Data_Capture.sw.dma_data_capture_regmap as dma_capture_regmap

# Load the overlay
overlay = Overlay('../../overlays/KR260_DMA_Capture/output/kr260_dma_capture.bit')

# Assign blocks to short vars
dma          = overlay.axi_dma_0

# Create PYNQ constructor for Consair RegMap
led_module   = led_regmap.RegMap(puch.PynqInterface(overlay.led_reg_0))
timestamp    = timestamp_regmap.RegMap(puch.PynqInterface(overlay.Timestamp_0))
dma_capture  = dma_capture_regmap.RegMap(puch.PynqInterface(overlay.DMA_Data_Capture_Top_0))

print("Done Init System")

Done Init System


##### Display Time Stamp Register
The time stamp is burned into the FPGA during the build process

In [3]:
print("FPGA Build Timestamp:  " + puch.get_timestamp_str(timestamp))

FPGA Build Timestamp:  2025/11/15 8:9:15


##### Toggle USER_LED[1:0] on/off

In [5]:
for i in range(16):
    led_module.user_leds = i%4
    time.sleep(1)

### Read DMA Data Capture Buffer

In [11]:
# Read the number of samples capture to use for DMA
Capture_Length       = 2**18
Enable_Debug_Cnt     = 1

# Reset DMA Capture Buffer and configure a capture
dma_capture.capture_length = Capture_Length
dma_capture.enable_debug_cnt = Enable_Debug_Cnt

dma_capture.fifo_flush = 1
print("DMA Capture Debug Enable      : " + str(dma_capture.enable_debug_cnt))
print("DMA Reset                     : " + str(dma_capture.fifo_flush))
dma_capture.fifo_flush = 0
print("DMA Reset                     : " + str(dma_capture.fifo_flush))

# Configure DMA Capture Buffer
print("DMA FIFO Depth                : " + str(dma_capture.max_depth))
print("DMA FIFO Capture Length       : " + str(dma_capture.capture_length))
print("DMA FIFO Write Pointer        : " + str(dma_capture.fifo_wr_ptr))
print("DMA FIFO Read Pointer         : " + str(dma_capture.fifo_rd_ptr))

# Trigger DMA Capture 
print("Triggered DMA Capture...")
dma_capture.capture_stb = 1

# Configure DMA Capture Buffer
print("DMA FIFO Depth                : " + str(dma_capture.max_depth))
print("DMA FIFO Capture Length       : " + str(dma_capture.capture_length))
print("DMA FIFO Write Pointer        : " + str(dma_capture.fifo_wr_ptr))
print("DMA FIFO Read Pointer         : " + str(dma_capture.fifo_rd_ptr))


# Allocate DMA Buffer and Read
start_time = time.time()
out_buffer = allocate(shape=(Capture_Length,), dtype=np.int32)
dma.recvchannel.transfer(out_buffer)
#print("Done tranfer out_buffer")
dma.recvchannel.wait()
stop_time = time.time()
print("Done Wait")
samples = out_buffer.astype(np.uint32)
print("Symbols in ARM Buffer         : " + str(len(samples)))
out_buffer.close()
del out_buffer

print("Execute Time                  : " + str(stop_time-start_time) + " sec")
print("DMA FIFO Write Pointer        : " + str(dma_capture.fifo_wr_ptr))
print("DMA FIFO Read Pointer         : " + str(dma_capture.fifo_rd_ptr))
dma_capture.fifo_flush = 1
print("DMA FIFO Write Pointer        : " + str(dma_capture.fifo_wr_ptr))
print("DMA FIFO Read Pointer         : " + str(dma_capture.fifo_rd_ptr))
plot_time(samples)


DMA Capture Debug Enable      : 1
DMA Reset                     : 1
DMA Reset                     : 0
DMA FIFO Depth                : 32768
DMA FIFO Capture Length       : 262144
DMA FIFO Write Pointer        : 0
DMA FIFO Read Pointer         : 0
Triggered DMA Capture...
DMA FIFO Depth                : 32768
DMA FIFO Capture Length       : 262144
DMA FIFO Write Pointer        : 32770
DMA FIFO Read Pointer         : 32770


RuntimeError: DMA channel not started

In [ ]:
dma.register_map

### Reset the DMA Controller
AXI DMA LogiCORE IP Product Guide: https://docs.amd.com/r/en-US/pg021_axi_dma/MM2S_DMACR-MM2S-DMA-Control-Register-Offset-00h

In [10]:
# Initiate the DMA Reset for both RD/WR
dma.register_map.MM2S_DMACR.Reset = 1
dma.register_map.S2MM_DMACR.Reset = 1

# Re-enable the DMA Channel
dma.register_map.MM2S_DMACR.RS = 1
dma.register_map.S2MM_DMACR.RS = 1

# Complete it again
dma.recvchannel.stop()
dma.sendchannel.stop()
dma.recvchannel.start()
dma.sendchannel.start()

# clear buffers
out_buffer.close()
del out_buffer

AttributeError: 'NoneType' object has no attribute 'stop'

In [ ]:
dma.recvchannel.idle
dma.sendchannel.running

In [ ]:
in_buffer.close()
out_buffer.close()